In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Substance_Use_Disorder"
cohort = "GSE161986"

# Input paths
in_trait_dir = "../../input/GEO/Substance_Use_Disorder"
in_cohort_dir = "../../input/GEO/Substance_Use_Disorder/GSE161986"

# Output paths
out_data_file = "../../output/preprocess/Substance_Use_Disorder/GSE161986.csv"
out_gene_data_file = "../../output/preprocess/Substance_Use_Disorder/gene_data/GSE161986.csv"
out_clinical_data_file = "../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE161986.csv"
json_path = "../../output/preprocess/Substance_Use_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Network preservation reveals shared and unique biological processes associated with chronic alcohol abuse in the NAc and PFC [mRNA]"
!Series_summary	"Chronic alcohol abuse has been linked to the disruption of executive function and allostatic conditioning of reward response dysregulation in the mesocorticolimbic pathway (MCL). Here, we analyzed genome-wide mRNA and miRNA expression from matched cases with alcohol dependence (AD) and controls (n=35) via gene network analysis to identify unique and shared biological processes dysregulated in the prefrontal cortex (PFC) and nucleus accumbens (NAc). We further investigated potential mRNA/miRNA interactions at the network and individual gene expression levels to identify the neurobiological mechanisms underlying AD in the brain. By using genotyped and imputed SNP data, we identified expression quantitative trait loci (eQTL) uncovering potential genetic regulatory elements for gene networks associated w

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset mentions mRNA expression in the title and summary
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (alcohol dependence):
# Key 1 contains 'diagnosis: Alcohol' and 'diagnosis: Control'
trait_row = 1

# For age:
# Key 2 contains age information
age_row = 2

# For gender:
# Key 3 contains 'Sex: Male', but there's only one value
# The dataset description also confirms "all male"
gender_row = None  # Only one value, not useful for association studies

# 2.2 Data Type Conversion

# Convert trait (alcohol dependence)
def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if "diagnosis:" in value:
        value = value.split("diagnosis:")[1].strip()
    
    if "alcohol" in value:
        return 1  # Case (alcohol dependent)
    elif "control" in value:
        return 0  # Control
    else:
        return None

# Convert age
def convert_age(value):
    if not isinstance(value, str):
        return None
    
    if "age:" in value:
        try:
            age = int(value.split("age:")[1].strip())
            return age  # Return age as a continuous value
        except (ValueError, IndexError):
            return None
    return None

# Convert gender function (not used but defined for completeness)
def convert_gender(value):
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if "sex:" in value:
        value = value.split("sex:")[1].strip()
    
    if "male" in value:
        return 1
    elif "female" in value:
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the function from the library
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM4929029': [1.0, 61.0], 'GSM4929030': [0.0, 44.0], 'GSM4929031': [0.0, 62.0], 'GSM4929032': [1.0, 56.0], 'GSM4929033': [0.0, 63.0], 'GSM4929034': [1.0, 42.0], 'GSM4929035': [0.0, 46.0], 'GSM4929036': [0.0, 56.0], 'GSM4929037': [1.0, 52.0], 'GSM4929038': [0.0, 43.0], 'GSM4929039': [1.0, 59.0], 'GSM4929040': [1.0, 56.0], 'GSM4929041': [1.0, 54.0], 'GSM4929042': [1.0, 46.0], 'GSM4929043': [1.0, 39.0], 'GSM4929044': [1.0, 73.0], 'GSM4929045': [0.0, 56.0], 'GSM4929046': [0.0, 50.0], 'GSM4929047': [1.0, 63.0], 'GSM4929048': [1.0, 50.0], 'GSM4929049': [1.0, 50.0], 'GSM4929050': [1.0, 51.0], 'GSM4929051': [1.0, 64.0], 'GSM4929052': [1.0, 55.0], 'GSM4929053': [0.0, 55.0], 'GSM4929054': [0.0, 47.0], 'GSM4929055': [0.0, 50.0], 'GSM4929056': [0.0, 55.0], 'GSM4929057': [1.0, 53.0], 'GSM4929058': [0.0, 82.0], 'GSM4929059': [0.0, 64.0], 'GSM4929060': [1.0, 73.0], 'GSM4929061': [0.0, 73.0], 'GSM4929062': [0.0, 57.0], 'GSM4929063': [0.0, 59.0]}
Clinical data saved to .

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 70
Header line: "ID_REF"	"GSM4929029"	"GSM4929030"	"GSM4929031"	"GSM4929032"	"GSM4929033"	"GSM4929034"	"GSM4929035"	"GSM4929036"	"GSM4929037"	"GSM4929038"	"GSM4929039"	"GSM4929040"	"GSM4929041"	"GSM4929042"	"GSM4929043"	"GSM4929044"	"GSM4929045"	"GSM4929046"	"GSM4929047"	"GSM4929048"	"GSM4929049"	"GSM4929050"	"GSM4929051"	"GSM4929052"	"GSM4929053"	"GSM4929054"	"GSM4929055"	"GSM4929056"	"GSM4929057"	"GSM4929058"	"GSM4929059"	"GSM4929060"	"GSM4929061"	"GSM4929062"	"GSM4929063"
First data line: "1007_s_at"	9.84717	9.44133	9.16609	9.11637	8.7745	8.65546	8.71848	8.83187	10.4673	8.6703	9.01476	9.49297	8.33806	9.37888	8.61347	8.9113	9.45797	9.08599	9.89926	9.07525	10.1801	10.005	9.59456	9.16801	9.18323	9.24158	8.61665	8.71702	8.73752	9.36581	9.11207	9.51819	8.97471	9.17735	9.03797
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '16002

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from this Affymetrix microarray dataset
# The IDs like "1007_s_at", "1053_at", etc. are Affymetrix probe set IDs
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE161986
Line 6: !Series_title = Network preservation reveals shared and unique biological processes associated with chronic alcohol abuse in the NAc and PFC [mRNA]
Line 7: !Series_geo_accession = GSE161986
Line 8: !Series_status = Public on Nov 24 2020
Line 9: !Series_submission_date = Nov 23 2020
Line 10: !Series_last_update_date = Nov 29 2022
Line 11: !Series_pubmed_id = 33332381
Line 12: !Series_summary = Chronic alcohol abuse has been linked to the disruption of executive function and allostatic conditioning of reward response dysregulation in the mesocorticolimbic pathway (MCL). Here, we analyzed genome-wide mRNA and miRNA expression from matched cases with alcohol dependence (AD) and controls (n=35)


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the columns in gene annotation data for mapping

# From the previous output, we can see that:
# - 'ID' is the column containing Affymetrix probe IDs like "1007_s_at" (same as in gene_data index)
# - 'Gene Symbol' is the column containing human gene symbols like "DDR1 /// MIR4640"

# 2. Create gene mapping dataframe by extracting the two relevant columns
mapping_df = gene_annotation[['ID', 'Gene Symbol']]
mapping_df = mapping_df.rename(columns={'Gene Symbol': 'Gene'})

# Preview the mapping dataframe
print("Preview of gene mapping dataframe:")
print(preview_df(mapping_df))

# 3. Apply gene mapping to convert probe-level data to gene expression data
# This will split probe values among genes when one probe maps to multiple genes,
# and then sum all probe contributions for each gene
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the gene expression data
print("\nPreview of gene expression data (after mapping):")
print(f"Shape: {gene_data.shape}")
print(f"First few gene symbols: {list(gene_data.index[:5])}")
print(preview_df(gene_data))

# Additionally, normalize gene symbols using the provided function
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nPreview of gene expression data (after normalization):")
print(f"Shape: {gene_data.shape}")
print(f"First few normalized gene symbols: {list(gene_data.index[:5])}")


Preview of gene mapping dataframe:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Preview of gene expression data (after mapping):
Shape: (13830, 35)
First few gene symbols: ['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS']
{'GSM4929029': [4.54833, 9.18608, 6.48012, 4.38932, 6.2499], 'GSM4929030': [5.02891, 8.9158, 6.39153, 4.20068, 6.20668], 'GSM4929031': [4.50991, 8.58664, 6.33478, 4.50872, 6.32526], 'GSM4929032': [4.59167, 9.40771, 6.48012, 4.37435, 5.91476], 'GSM4929033': [4.48148, 8.99435, 6.28686, 4.39409, 6.21621], 'GSM4929034': [4.45979, 7.56797, 6.67685, 4.35462, 6.23956], 'GSM4929035': [4.74628, 8.93715, 6.6725, 4.37792, 5.98471], 'GSM4929036': [4.87043, 9.13947, 6.50615, 4.4689, 6.45009], 'GSM4929037': [4.78021, 10.3, 6.48012, 4.4701, 6.44977], 'GSM4929038': [5.00795, 9.41105, 6.86503, 4.45795, 6.40924], 'GSM4929039': [4.55028, 7.55479, 6.627, 4.44392, 6.38081], 'GSM4929040': [4.90808, 8.49848, 6.8

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains gene expression data for {trait} analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (13542, 35)


Gene data shape after normalization: (13542, 35)


Normalized gene data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE161986.csv
Raw clinical data shape: (13, 36)
Clinical features:
                        GSM4929029  GSM4929030  GSM4929031  GSM4929032  \
Substance_Use_Disorder         1.0         0.0         0.0         1.0   
Age                           61.0        44.0        62.0        56.0   

                        GSM4929033  GSM4929034  GSM4929035  GSM4929036  \
Substance_Use_Disorder         0.0         1.0         0.0         0.0   
Age                           63.0        42.0        46.0        56.0   

                        GSM4929037  GSM4929038  ...  GSM4929054  GSM4929055  \
Substance_Use_Disorder         1.0         0.0  ...         0.0         0.0   
Age                           52.0        43.0  ...        47.0        50.0   

                        GSM4929056  GSM4929057  GSM4929058  GSM4929059  \
Substance_Use_Disorder         0.0         1.0         0.0         0.0   
Age           

Data shape after handling missing values: (35, 13544)
For the feature 'Substance_Use_Disorder', the least common label is '0.0' with 17 occurrences. This represents 48.57% of the dataset.
The distribution of the feature 'Substance_Use_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 50.0
  50% (Median): 55.0
  75%: 61.5
Min: 39.0
Max: 82.0
The distribution of the feature 'Age' in this dataset is fine.



Linked data saved to ../../output/preprocess/Substance_Use_Disorder/GSE161986.csv
